In [1]:
import xarray as xr
import pickle
import math
from pyproj import CRS, Transformer
import pyproj
import numpy as np
import xesmf as xe

ModuleNotFoundError: No module named 'xesmf'

In [ ]:
def calc_heading(lat1, lon1, lat2, lon2):

    lat1_rad = np.deg2rad(lat1)
    lat2_rad = np.deg2rad(lat2)
    lon1_rad = np.deg2rad(lon1)
    lon2_rad = np.deg2rad(lon2)

    d_lon = lon2_rad - lon1_rad

    y = np.sin(d_lon) * np.cos(lat2_rad)
    x = (np.cos(lat1_rad) * np.sin(lat2_rad) -
         np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(d_lon)) 
    
    initial_bearing_rad = np.arctan2(y, x)

    initial_bearing_deg = np.rad2deg(initial_bearing_rad)

    bearing = (initial_bearing_deg + 360) % 360

    return bearing 


In [ ]:
calc_heading(30, -100, 35, -110)

In [ ]:
def to_polar(
        ds: xr.Dataset,
        origin_lat: float,
        origin_lon: float,
        ref_lat: float,
        ref_lon: float,
        rho_steps: int,
        theta_steps: int
    ) -> xr.Dataset:
    
        R = 6378137 # radius of Earth in meters
        R_max = 300.0 # Max range in km
        R_step = 2.0  # Range step in km
        A_step = 1.0  # Azimuth step in degrees

        new_range = np.arange(0.0, R_max + R_step, R_step)
        new_azimuth = np.arange(0.0, 360.0, A_step)
        R, T = np.meshgrid(new_azimuth, new_range, indexing = 'ij')

        heading_shift = calc_heading(origin_lat, origin_lon, ref_lat, ref_lon)
        local_proj = pyproj.Proj(
                                proj='stere',        
                                lat_0=origin_lat,         # Center projection at storm center lat
                                lon_0=origin_lon,         # Set central meridian (0 heading) to storm center lon
                                lat_ts=origin_lat,        # Latitude of true scale set near the center
                                a=6371000.0,         # Earth Radius (e.g., Sphere)
                                units='m'            # X/Y in meters
                                )
        
        X = R * np.cos(T) * 1000 # Convert km to meters
        Y = R * np.sin(T) * 1000
        local_proj_str = f"+proj=aeqd +lat_0={origin_lat} +lon_0={origin_lon} +units=m"
        transformer = pyproj.Transformer.from_crs(local_proj_str, "EPSG:4326", always_xy=True)

        target_lons, target_lats = transformer.transform(X, Y)


        ds_out = xr.Dataset(
                            coords={
                                    "latitude": (("radius", "angle"), target_lats),
                                    "longitude": (("radius", "angle"), target_lons),
                                    "radius": new_range,
                                    "angle": new_azimuth
                                        }
                            )
        
        regridder = xe.Regridder(ds, ds_out,'bilinear')
        Lon_in, Lat_in = ds['longitude'].values, ds['latitude'].values
        Lon_mesh, Lat_mesh = np.meshgrid(Lon_in, Lat_in)
        X_local, Y_local = local_proj(Lon_mesh, Lat_mesh)

        print(np.shape(X_local))
        print(ds.dims)
        X_da = xr.DataArray(X_local, coords=ds.coords, dims=ds.dims)
        Y_da = xr.DataArray(Y_local, coords=ds.coords, dims=ds.dims)

        R_da = np.sqrt(X_da**2 + Y_da**2) / 1000.0

        Theta_rad = np.arctan2(Y_da, X_da)

        Theta_deg = np.degrees(Theta_rad)

        Theta_North = 90.0 - Theta_deg
        Theta_North = np.mod(Theta_North, 360.0) 

        Theta_arbitrary = Theta_North - heading_shift
        Theta_arbitrary = np.mod(Theta_arbitrary, 360.0)

     
        
        
        da_polar_grid = ds.interp(
        # The new dimensions (on the left) are mapped to the 2D auxiliary coordinates (on the right)
                                  range=(R_da, new_range), 
                                  azimuth=(Theta_arbitrary, new_azimuth),
                                  method="linear" # Choose your interpolation method (e.g., 'linear', 'nearest')
                                  )
        
        
        
        return da_polar_grid



    

        

    

In [ ]:
drop_variables = ['q', 'w', 'wz', 'absv', 'clwmr', 'icmr', 'rwmr', 'snmr','grle', 'rare']
ds_atm = xr.open_dataset('Data\92l.2023082400.hfsa.storm.atm.f024.grb2',
                         drop_variables = drop_variables, 
                         filter_by_keys={'typeOfLevel': 'isobaricInhPa'})

ds_sfc = xr.open_dataset('Data\92l.2023082400.hfsa.storm.atm.f024.grb2',
                         drop_variables = drop_variables,  
                         filter_by_keys={'typeOfLevel': 'meanSea'})

In [ ]:
ds_sfc

In [ ]:
c_msp = ds_sfc['prmsl'].min()
min_coords = ds_sfc['prmsl'].where(ds_sfc['prmsl'] == c_msp, drop=True).squeeze()

In [ ]:
len(ds_sfc.longitude.values)

In [ ]:
min_coords

In [ ]:
pol_ds = to_polar(ds = ds_sfc.isel(longitude = slice(100,-100)).squeeze(), origin_lat = 17.18, origin_lon = 319.9, ref_lat = 17.5, ref_lon= 317.0, theta_steps = 720, rho_steps = 1100 )

In [ ]:
pol_ds


In [ ]:
ds_sfc.latitude

In [ ]:
landmask = regionmask.defined_regions.natural_earth.land_110

In [ ]:
ds_atm.sel(isobaricInhPa = cmp, method = 'nearest')

In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

def list_s3_files(bucket_name, prefix=''):
    """
    Lists files in an AWS S3 bucket, optionally filtered by a prefix.

    Args:
        bucket_name (str): The name of the S3 bucket.
        prefix (str, optional): An optional prefix to filter files (e.g., 'folder/').
    """
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    
    # Handle pagination for more than 1000 objects
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    file_list = []
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                file_list.append(obj)
        else:
            print(f"No files found in '{bucket_name}' with prefix '{prefix}'.")
    return file_list

In [ ]:
list = list_s3_files('noaa-nws-hafs-pds', prefix = 'hfsa')

In [ ]:
with open('link_list.pkl', 'wb') as f:
    pickle.dump(list, f)

In [ ]:
with open('Data/links/link_list.txt', 'w') as f:
    for item in list:
        if ('f024' in item['Key'] or 'f027' in item['Key']) and 'idx' not in item['Key']:
            f.write(item['Key'] + '\n')

In [ ]:
list